# This is the TopicCategory classifier for outbreak.info publications. 

It has not yet been tested for non-publications as it is trained completely on publication data primarily from LitCovid. It contains functions for the following tasks:

1. Retrieve and format LitCovid Topics (must be done frequently)
2. Build Behavioral and offtopic training sets (must be done frequently)
3. Generate Models (should be done rarely, or only on newly introduced topics)
4. Classify non-litcovid publications using models (must be done for new publications)
5. Merge formatted LitCovid Topics and Offtopic data with predicted data for inclusion into all publications

In [1]:
%%time
#### Update Litcovid topics
import os
import pathlib
from src.fetch_litcovid_topics import *
    
#script_path = pathlib.Path(__file__).parent.absolute()
script_path = ''
DATAPATH = os.path.join(script_path,'data/')

get_litcovid_topics(DATAPATH)

## Runtime: 43.7 seconds
## Runtime (2022.09.07) - 1 min 22 s

Wall time: 1min 22s


In [3]:
%%time
####  Update Offtopics
import os
import pathlib
from src.fetch_offtopics import *
#### MAIN
#script_path = pathlib.Path(__file__).parent.absolute()
script_path = ''
DATAPATH = os.path.join(script_path,'data/')
RESULTSPATH = os.path.join(script_path,'results/')

get_other_topics(DATAPATH,RESULTSPATH)

## Runtime: 21 min
## Runtime (2022.09.07 - 31min 53s)

Wall time: 31min 53s


In [4]:
%%time
#### Update clinical topics
import os
import pathlib
from src.fetch_clinical_trials import *
#### MAIN
#script_path = pathlib.Path(__file__).parent.absolute()
script_path = ''
DATAPATH = os.path.join(script_path,'data/')
RESULTSPATH = os.path.join(script_path,'results/')
SUBDATAPATH = os.path.join(DATAPATH,'subtopics/')
CLINDATAPATH = os.path.join(SUBDATAPATH,'ct_topics/')

update_clin_cats(DATAPATH,CLINDATAPATH)
## Runtime (2022.09.07 - 5 min 51 s)

Wall time: 5min 51s


In [1]:
%%time
#### Update subtopics topics
import os
import pathlib
from src.fetch_subtopics import *
from src.fetch_litsubtopics import *
#### MAIN
#script_path = pathlib.Path(__file__).parent.absolute()
script_path = ''
DATAPATH = os.path.join(script_path,'data/')
RESULTSPATH = os.path.join(script_path,'results/')
SUBDATAPATH = os.path.join(DATAPATH,'subtopics/')

get_sub_topics(DATAPATH,RESULTSPATH)
## Runtime (2022.09.09 - 1h 4min 13s)

Wall time: 1h 4min 13s


In [3]:
%%time
## Generate histogram of litcovid topics, other topics, and clinical cats
import pandas as pd
import os
from pandas import read_csv

script_path = ''
DATAPATH = os.path.join(script_path,'data/')

littopics = read_csv(os.path.join(DATAPATH,'litcovidtopics.tsv'), delimiter='\t', header=0, index_col=0)
print(littopics.groupby('topicCategory').size().reset_index(name="counts"))

offtopics = read_csv(os.path.join(DATAPATH,'othertopics.tsv'), delimiter='\t', header=0, index_col=0)
print(offtopics.groupby('topicCategory').size().reset_index(name="counts"))

subtopics = read_csv(os.path.join(DATAPATH,'subtopics.tsv'), delimiter='\t', header=0, index_col=0)
print(subtopics.groupby('topicCategory').size().reset_index(name="counts"))

       topicCategory  counts
0  Case Descriptions   13871
1          Diagnosis   46859
2        Forecasting    3029
3          Mechanism   31912
4         Prevention   68908
5       Transmission    6872
6          Treatment   67460
          topicCategory  counts
0   Behavioral Research   53557
1              Clinical    4263
2           Environment    6884
3          Epidemiology   16296
4  Information Sciences   11513
5          Risk Factors    5119
                   topicCategory  counts
0             Antibody Detection    8262
1                      Biologics    6107
2         Classical Epidemiology   22484
3                   Host Factors     686
4   Host-Intermediate Reservoirs    1248
5         Immunological Response   11433
6          Individual Prevention   64223
7         Mechanism of Infection    5468
8      Mechanism of Transmission     829
9                   Medical Care    5804
10        Molecular Epidemiology     441
11           Pathology-Radiology    9224
12     Phar

In [ ]:
#### Run classifier tests
import os
import pandas as pd
import pathlib
from src.train_classifier import *

#script_path = pathlib.Path(__file__).parent.absolute()
script_path = ''
DATAPATH = os.path.join(script_path,'data/')
RESULTSPATH = os.path.join(script_path,'results/')
littopicsfile = os.path.join(DATAPATH,'litcovidtopics.tsv')
offtopicsfile = os.path.join(DATAPATH,'othertopics.tsv')
littopicsdf = read_csv(littopicsfile,delimiter='\t',header=0,index_col=0)
offtopicsdf = read_csv(offtopicsfile,delimiter='\t',header=0,index_col=0)
topicsdf = pd.concat((littopicsdf,offtopicsdf),ignore_index=True)
topiclist = topicsdf['topicCategory'].unique().tolist()

testresultsdf = run_test(RESULTSPATH,topicsdf,classifierset_type='best',export_report=True)

In [ ]:
#### Update all models
import os
import pandas as pd
import pathlib
from src.train_classifier import *

#script_path = pathlib.Path(__file__).parent.absolute()
script_path = ''
DATAPATH = os.path.join(script_path,'data/')
RESULTSPATH = os.path.join(script_path,'results/')
MODELPATH = os.path.join(script_path,'models/')

littopicsfile = os.path.join(DATAPATH,'litcovidtopics.tsv')
offtopicsfile = os.path.join(DATAPATH,'othertopics.tsv')
littopicsdf = read_csv(littopicsfile,delimiter='\t',header=0,index_col=0)
offtopicsdf = read_csv(offtopicsfile,delimiter='\t',header=0,index_col=0)
topicsdf = pd.concat((littopicsdf,offtopicsdf),ignore_index=True)
topiclist = topicsdf['topicCategory'].unique().tolist()    

classifiers = load_classifiers('best')
generate_models(MODELPATH,topicsdf,classifiers)

In [ ]:
#### Update single topic model
import os
import pandas as pd
import pathlib
from src.train_classifier import *

#script_path = pathlib.Path(__file__).parent.absolute()
script_path = ''
DATAPATH = os.path.join(script_path,'data/')
RESULTSPATH = os.path.join(script_path,'results/')
MODELPATH = os.path.join(script_path,'models/')

littopicsfile = os.path.join(DATAPATH,'litcovidtopics.tsv')
offtopicsfile = os.path.join(DATAPATH,'othertopics.tsv')
littopicsdf = read_csv(littopicsfile,delimiter='\t',header=0,index_col=0)
offtopicsdf = read_csv(offtopicsfile,delimiter='\t',header=0,index_col=0)
topicsdf = pd.concat((littopicsdf,offtopicsdf),ignore_index=True)
topiclist = topicsdf['topicCategory'].unique().tolist()    

classifiers = load_classifiers('best')
topic_to_check = input("enter the topic Category: ")
generate_models(MODELPATH,topicsdf,classifiers,topic_to_check)

In [ ]:
#### Update annotations for all pubs
import os
import pathlib

from src.classify_pubs import *
from src.common import load_classifiers
#### MAIN
#script_path = pathlib.Path(__file__).parent.absolute()
script_path = ''
DATAPATH = os.path.join(script_path,'data/')
RESULTSPATH = os.path.join(script_path,'results/')
MODELPATH = os.path.join(script_path,'models/')
PREDICTPATH = os.path.join(script_path,'predictions/')

littopicsfile = os.path.join(DATAPATH,'litcovidtopics.tsv')
offtopicsfile = os.path.join(DATAPATH,'othertopics.tsv')
littopicsdf = read_csv(littopicsfile,delimiter='\t',header=0,index_col=0)
offtopicsdf = read_csv(offtopicsfile,delimiter='\t',header=0,index_col=0)
topicsdf = pd.concat((littopicsdf,offtopicsdf),ignore_index=True)
topiclist = topicsdf['topicCategory'].unique().tolist()

classifiers = load_classifiers('best')
load_annotations(MODELPATH,PREDICTPATH,RESULTSPATH,topicsdf,classifiers,newonly = False)

In [ ]:
#### Update annotations for new pubs only
import os
import pathlib

from src.classify_pubs import *
from src.common import load_classifiers
#### MAIN
#script_path = pathlib.Path(__file__).parent.absolute()
script_path = ''
DATAPATH = os.path.join(script_path,'data/')
RESULTSPATH = os.path.join(script_path,'results/')
MODELPATH = os.path.join(script_path,'models/')
PREDICTPATH = os.path.join(script_path,'predictions/')

littopicsfile = os.path.join(DATAPATH,'litcovidtopics.tsv')
offtopicsfile = os.path.join(DATAPATH,'othertopics.tsv')
littopicsdf = read_csv(littopicsfile,delimiter='\t',header=0,index_col=0)
offtopicsdf = read_csv(offtopicsfile,delimiter='\t',header=0,index_col=0)
topicsdf = pd.concat((littopicsdf,offtopicsdf),ignore_index=True)
topiclist = topicsdf['topicCategory'].unique().tolist()

classifiers = load_classifiers('best')
load_annotations(MODELPATH,PREDICTPATH,RESULTSPATH,topicsdf,classifiers)

In [13]:
%%time
#### Update (refresh) the annotations for all publications
import os
import pathlib

from src.classify_pubs import *
from src.common import load_classifiers
#### MAIN
#script_path = pathlib.Path(__file__).parent.absolute()
script_path = ''
DATAPATH = os.path.join(script_path,'data/')
RESULTSPATH = os.path.join(script_path,'results/')
MODELPATH = os.path.join(script_path,'models/')
PREDICTPATH = os.path.join(script_path,'predictions/')

#### Refresh the classification of other resources
littopicsfile = os.path.join(DATAPATH,'litcovidtopics.tsv')
offtopicsfile = os.path.join(DATAPATH,'othertopics.tsv')
subtopicsfile = os.path.join(RESULTSPATH,'subtopicCats.tsv')
littopicsdf = read_csv(littopicsfile,delimiter='\t',header=0,index_col=0)
offtopicsdf = read_csv(offtopicsfile,delimiter='\t',header=0,index_col=0)
subtopicsdf = read_csv(subtopicsfile,delimiter='\t',header=0,index_col=0)
topicsdf = pd.concat((littopicsdf,offtopicsdf,subtopicsdf),ignore_index=True)
topicsdf.drop_duplicates(keep='first',inplace=True)

classifiers = load_classifiers('best')
load_annotations(DATAPATH,MODELPATH,PREDICTPATH,RESULTSPATH,topicsdf,classifiers,False)

##Run time: 36min 54s

Wall time: 36min 45s


#### Run times for different parts of cell above

In [2]:
%%time
#### Update (refresh) the annotations for all publications
import os
import pathlib
from src.classify_pubs import *
from src.common import load_classifiers
from src.common import topic_dict
#### MAIN
#script_path = pathlib.Path(__file__).parent.absolute()
script_path = ''
DATAPATH = os.path.join(script_path,'data/')
RESULTSPATH = os.path.join(script_path,'results/')
MODELPATH = os.path.join(script_path,'models/')
PREDICTPATH = os.path.join(script_path,'predictions/')

classifiers = load_classifiers('best')
classifierlist = classifiers.keys()
CLINPREDICTPATH = os.path.join(PREDICTPATH,'clinpredict/')
PUBPREDICTPATH = os.path.join(PREDICTPATH,'pubpredict/')
classify_clins(DATAPATH,MODELPATH,PREDICTPATH,classifiers,topic_dict)
clin_total_agree = merge_predictions(CLINPREDICTPATH,topic_dict,classifierlist,agreetype='perfect')
all_ids = get_pub_ids(sourceset="nonlitcovid")
print(len(all_ids))

##Run time 8min 41s for 28372 docs

Wall time: 8min 41s


In [8]:
%%time
classify_pubs(MODELPATH,PUBPREDICTPATH,all_ids,topic_dict,classifiers,False)
total_agree = merge_predictions(PUBPREDICTPATH,topic_dict,classifierlist,'perfect')
totalresults = pd.concat((total_agree,topicsdf,clin_total_agree),ignore_index=True)
print(len(totalresults))
print(totalresults.head(n=2))

##Run time for 28372 docs: 28min 19s, resulting in 799430 

799430
                   _id         topicCategory
0  2020.01.31.20019935  Information Sciences
1    2020.02.03.932350  Information Sciences
Wall time: 65 ms


In [10]:
%%time
allresults = include_clin(totalresults)
allresults['topicCategory'] = allresults['topicCategory'].str.replace('-','/')
allresults.dropna(axis=0,inplace=True)
allresults.drop_duplicates(keep='first',inplace=True)
allresults.reset_index(drop=True)
cleanresults = clean_results(allresults) 

print(len(allresults))
print(allresults.head(n=2))
print(len(cleanresults))
print(cleanresults.head(n=2))
##Run time 21.2s

821890
           _id topicCategory
145231  092619     Mechanism
54831   105437   Forecasting
254411
      _id                                      topicCategory
0  092619                                        [Mechanism]
1  105437  [Forecasting, Mechanism, Molecular Epidemiolog...
Wall time: 21.1 s


In [11]:
%%time
cleanresults.to_csv(os.path.join(RESULTSPATH,'topicCats.tsv'),mode='w',sep='\t',header=True)
updated_results = read_csv(os.path.join(RESULTSPATH,'topicCats.tsv'),delimiter='\t',header=0,index_col=0)
updated_results.drop_duplicates(subset='_id',keep='last',inplace=True)
updated_results.to_csv(os.path.join(RESULTSPATH,'topicCats.tsv'),sep='\t',header=True)
updated_results.to_json(os.path.join(RESULTSPATH,'topicCats.json'), orient='records')

##Run time 3.14 s

Wall time: 3.14 s


## Increasing the efficiency of common functions

In [ ]:
%%time
#### Increasing the efficiency of the cleanresults function

import os
import pandas as pd
from pandas import read_csv

script_path = ''
DATAPATH = os.path.join(script_path,'data/')
RESULTSPATH = os.path.join(script_path,'results/')
MODELPATH = os.path.join(script_path,'models/')
PREDICTPATH = os.path.join(script_path,'predictions/')

littopicsfile = os.path.join(DATAPATH,'litcovidtopics.tsv')
offtopicsfile = os.path.join(DATAPATH,'othertopics.tsv')
subtopicsfile = os.path.join(DATAPATH,'subtopics.tsv')
littopicsdf = read_csv(littopicsfile,delimiter='\t',header=0,index_col=0)
offtopicsdf = read_csv(offtopicsfile,delimiter='\t',header=0,index_col=0)
subtopicsdf = read_csv(subtopicsfile,delimiter='\t',header=0,index_col=0)
subtopic_results = read_csv(os.path.join(RESULTSPATH,'subtopicCats.tsv'),delimiter='\t',header=0,index_col=0)
topicsdf = pd.concat((littopicsdf,offtopicsdf,subtopicsdf,subtopic_results),ignore_index=True)


def clean_results(allresults):
    cleanresults = allresults.groupby('_id')['topicCategory'].apply(list).reset_index(name='newTopicCategory')
    cleanresults.rename(columns={'newTopicCategory':'topicCategory'},inplace=True)
    return(cleanresults) 

cleanresults = clean_results(topicsdf)
print(cleanresults.head(n=2))

#### The new method takes 14.5 seconds

In [ ]:
%%time
#### Increasing the efficiency of the old cleanresults function

def clean_results(allresults):
    allresults.drop_duplicates(keep="first",inplace=True)
    counts = allresults.groupby('_id').size().reset_index(name='counts')
    duplicates = counts.loc[counts['counts']>1]
    singles = counts.loc[counts['counts']==1]
    dupids = duplicates['_id'].unique().tolist()
    tmplist = []
    for eachid in dupids:
        catlist = allresults['topicCategory'].loc[allresults['_id']==eachid].tolist()
        tmplist.append({'_id':eachid,'topicCategory':catlist})
    tmpdf = pd.DataFrame(tmplist)  
    tmpsingledf = allresults[['_id','topicCategory']].loc[allresults['_id'].isin(singles['_id'].tolist())]
    idlist = tmpsingledf['_id'].tolist()
    catlist = tmpsingledf['topicCategory'].tolist()
    cattycat = [[x] for x in catlist]
    list_of_tuples = list(zip(idlist,cattycat))
    singledf = pd.DataFrame(list_of_tuples, columns = ['_id', 'topicCategory']) 
    cleanresults = pd.concat((tmpdf,singledf),ignore_index=True)
    return(cleanresults)

cleanresults = clean_results(topicsdf)
print(cleanresults.head(n=2))

#### The old method took a ridiculously long time! (over 30 min)

In [ ]:
%%time
#### The old get agreement function

import os
import pandas as pd
from pandas import read_csv
from src.common import load_classifiers

def get_agreement(PREDICTPATH,eachtopic,classifierlist):
    agreement = pd.DataFrame(columns=['_id','topicCategory','pos_pred_count','pos_pred_algorithms'])
    classresult = pd.DataFrame(columns=['_id','prediction','topicCategory','classifier'])
    for eachclass in classifierlist:
        tmpfile = read_csv(os.path.join(PREDICTPATH,eachtopic+"_"+eachclass+".tsv"),delimiter='\t',header=0,index_col=0)
        classresult = pd.concat((classresult,tmpfile),ignore_index=True)
    posresults = classresult.loc[classresult['prediction']=='in category']
    agreecounts = posresults.groupby('_id').size().reset_index(name='counts')
    no_agree = posresults.loc[posresults['_id'].isin(agreecounts['_id'].loc[agreecounts['counts']==1].tolist())].copy()
    no_agree.rename(columns={'classifier':'pos_pred_algorithms'},inplace=True)
    no_agree['pos_pred_count']=1
    no_agree.drop('prediction',axis=1,inplace=True)
    perfect_agree = posresults.loc[posresults['_id'].isin(agreecounts['_id'].loc[agreecounts['counts']==len(classifierlist)].tolist())].copy()
    perfect_agree['pos_pred_count']=len(classifierlist)
    perfect_agree['pos_pred_algorithms']=str(classifierlist)
    perfect_agree.drop(['prediction','classifier'],axis=1,inplace=True)
    perfect_agree.drop_duplicates('_id',keep='first',inplace=True)
    partialcountids = agreecounts['_id'].loc[((agreecounts['counts']>1)&
                                          (agreecounts['counts']<len(classifierlist)))].tolist()
    tmplist = []
    for eachid in list(set(partialcountids)):
        tmpdf = posresults.loc[posresults['_id']==eachid]
        tmpdict = {'_id':eachid,'topicCategory':eachtopic,'pos_pred_count':len(tmpdf),
                   'pos_pred_algorithms':str(tmpdf['classifier'].tolist())}
        tmplist.append(tmpdict)
    partial_agree = pd.DataFrame(tmplist)    
    agreement = pd.concat((agreement,no_agree,partial_agree,perfect_agree),ignore_index=True)
    return(agreement)

classifiers = load_classifiers('best')
classifierlist = list(classifiers.keys())
PREDICTPATH = os.path.join(script_path,'predictions/')
CLINPREDICTPATH = os.path.join(PREDICTPATH,'clinpredict/')
PUBPREDICTPATH = os.path.join(PREDICTPATH,'pubpredict/')
agreement = get_agreement(PUBPREDICTPATH,'Mechanism',classifierlist)
print(agreement.tail(n=2))

#### A test run for the Mechanism topic took 54 seconds to run

In [ ]:
%%time
#### increasing the efficiency of the get agreement function
import os
import pandas as pd
from pandas import read_csv
from src.common import load_classifiers

def get_agreement(PREDICTPATH,eachtopic,classifierlist):
    classresult = pd.DataFrame(columns=['_id','prediction','topicCategory','classifier'])
    for eachclass in classifierlist:
        tmpfile = read_csv(os.path.join(PREDICTPATH,eachtopic+"_"+eachclass+".tsv"),delimiter='\t',header=0,index_col=0)
        classresult = pd.concat((classresult,tmpfile),ignore_index=True)
    classresult.drop_duplicates(keep='first',inplace=True)
    posresults = classresult.loc[classresult['prediction']=='in category']
    agreement = posresults.groupby(['_id','topicCategory'])['classifier'].apply(list).reset_index(name='pos_pred_algorithms')
    agreement['pos_pred_count'] = agreecounts['pos_pred_algorithms'].str.len()
    return(agreement)


classifiers = load_classifiers('best')
classifierlist = classifiers.keys()
PREDICTPATH = os.path.join(script_path,'predictions/')
CLINPREDICTPATH = os.path.join(PREDICTPATH,'clinpredict/')
PUBPREDICTPATH = os.path.join(PREDICTPATH,'pubpredict/')
agreement = get_agreement(PUBPREDICTPATH,'Mechanism',classifierlist)
print(agreement.tail(n=2))
#### A test run for Mechanism took to run 5 seconds

In [ ]:
import os
import pathlib
import requests
import json
import pandas as pd
import pickle

from src.common import *
#### MAIN
#script_path = pathlib.Path(__file__).parent.absolute()
script_path = ''
DATAPATH = os.path.join(script_path,'data/')
RESULTSPATH = os.path.join(script_path,'results/')
MODELPATH = os.path.join(script_path,'models/')
PREDICTPATH = os.path.join(script_path,'predictions/')



WIKIDATAPATH = os.path.join(DATAPATH,'from wikidata/')
repurposetypes = ['Q12140', 'Q35456', 'Q28885102','Q8386']
headers = {'User-Agent': 'outbreak resource topic classifier bot (https://outbreak.info/; help@outbreak.info)'}
querystart = """
SELECT
  ?item ?itemLabel ?itemAltLabel
  ?value 
WHERE 
{
  ?item wdt:P31 wd:"""
queryend = """.        
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
repurpose = pd.DataFrame(columns=['wdid','drug_name','name','alias'])
for eachwdid in repurposetypes:
    query = querystart+eachwdid+queryend
    params = {'format': 'json', 'query': query, 'headers': headers}
    r = make_request(params)
    if r != 0:
        data = r.json()
        with open(os.path.join(WIKIDATAPATH,eachwdid+'.pickle'),'wb') as dumpfile:
            pickle.dump(data,dumpfile)
    else:
        with open(os.path.join(WIKIDATAPATH,eachwdid+'.pickle'),'rb') as loadfile:
            data = pickle.load(loadfile)  
    tmpdf = parse_wikidata(data)
    repurpose = pd.concat((repurpose,tmpdf),ignore_index=True)
    time.sleep(1)
repurpose.drop_duplicates(keep='first',inplace=True)
